## Tracking the eastward positive precipitation anomaly

The present method follows the technique of zhang and ling(2017) to identify the MJO events tracking the precipitation anomaly using TRMM datasets.
The method follows three basics steps

## Functions :

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset,num2date
from scipy.interpolate import RectBivariateSpline

####################### Function to convert meter/second to day/degree longitude or slope range##########

def convert_day_per_lon(v):  ## meter/second
    
    v_day      = v*3600*24/(100*1000)  # degree/day
    #  m (slope) = 1/velocity   #celocity=(slope in t-x plane)#
    m_day_deg  =  round(1./v_day,2)   # rounding upto second decimal
    
    return m_day_deg



################################# Function to generate trial lines:#################################

def trial_lines(x_range,m_range,x_org,time):
    
### x_range : Longitude range
### m_range : slope of the trial lines , note here slope is 1/m as timw y axis
### x_org   : 89.875 E
### y_org   : each time steps
### time_steps : Number of time steps

### output:
### 3D matrix yfit = [slope,predicted_time_ind,time_org/y_org]
    
    y_org     = time

    yfit      = np.zeros((len(x_range),len(m_range),len(y_org)))
    xfit      = np.zeros((len(x_range),len(m_range),len(y_org)))
    
    for t in np.arange(len(y_org)):  
        for i in np.arange(len(m_range)):    
            ytemp             =  m_range[i]*(x_range-x_org)+y_org[t]
            ytemp[ytemp<0.0]  = np.nan
            yfit[:,i,t]       = ytemp
            xfit[:,i,t]       = x_range
    
    return xfit, yfit
        
    
    
#######################function to retain only longest segment ###############################################

def retain_longest_segment(mask,gap):
    istrt = 0
    iend  = mask.shape[0]-1
    imask = mask.astype(int)
    
    for t in np.arange(mask.shape[2]):
        print(t)
        for m in np.arange(mask.shape[1]):
            #do nothing if all values are 1
            if all(mask[:,m,t]):
                continue
                
            #do nothing if all values are 0    
            if not any(mask[:,m,t]):
                continue
                
            temp = imask[:,m,t]
            df   = pd.DataFrame(list(temp),columns=['A'])
            df['block'] = (df.A.shift(1) != df.A).astype(int).cumsum()
            B = df.reset_index().groupby(['A','block'])['index'].apply(np.array)
            
            for k in np.arange(B[0].shape[0]):
                zlocs=B[0].iloc[k]
                #do nothing if zero segment occurs at the start or at the end
                if any(zlocs==istrt) or any(zlocs==iend):
                    continue
                    
                #replace with 1 if length zero segment is less than gap
                if len(zlocs)<gap:
                    temp[zlocs] = 1
            
            df   = pd.DataFrame(list(temp),columns=['A'])
            df['block'] = (df.A.shift(1) != df.A).astype(int).cumsum()
            B=df.reset_index().groupby(['A','block'])['index'].apply(np.array)
            
            #if only one segment is there, save it back to imask and go to next iteration
            if B[1].shape[0]==1:
                imask[:,m,t]=temp
                continue
            
            #if more than one segment is there, then retain only the longest segment
            zlen = len(B[1].iloc[0])
            kmax = 0
            for k in np.arange(1,B[1].shape[0]):
                if zlen>len(B[1].iloc[k]):
                    zlen = len(B[1].iloc[k])
                    kmax = k
                    
            zlocs = B[1].iloc[kmax]
            temp[:] = 0
            temp[zlocs] = 1
            imask[:,m,t] = temp
    return imask




## Processing

In [ ]:
dx            = 0.25 #(in degrees)
dt            = 1. #(in days)
vmin          = 2. # (in m/sec)
vmax          = 10. # (in m/sec)
nlines        = 20
x_org         = 89.875
gap_threshold = 10. #(in degrees)

m2            = convert_day_per_lon(vmin)
m1            = convert_day_per_lon(vmax)
m_range       = np.linspace(m1,m2,nlines)

gap           = int(gap_threshold/dx)

dataset       = Dataset('MJO_filtered_full.nc')
#dataset.variables.keys()

var           = dataset.variables['mjo_time_lon'][:]
lon           = dataset.variables['lon'][:]
time          = dataset.variables['time'][:]

#Initialize interpolation
interp_spline = RectBivariateSpline(time, lon, var)

#Initialize trial lines
tlx, tly      = trial_lines(lon,m_range,x_org,time)

#Do interpolation onto trial lines

P           = interp_spline(tly,tlx,grid=False)

print(P.shape)

In [ ]:
sigma = np.std(var,0)
plt.plot(sigma)
mask  = numpy.full(P.shape, False)

for i in np.arange(sigma.size):
    mask[i,:,:] = P[i,:,:]>=sigma[i]

imask = retain_longest_segment(mask,gap)

In [ ]:
A = np.nansum(P*imask,0)
plt.contourf((1./m_range),time[0:120],A[:,0:120].T)
plt.colorbar();

np.count_nonzero(imask[:,0,85])

In [ ]:
y1=tly[:,0,0]
x1=tlx[:,0,0]
plt.plot(x1,y1)
ym=y1*imask[:,0,0]
plt.plot(x1,ym,'r')


In [ ]:
#plt.plot(lon,(P[:,0,0]>=sigma).astype(int))
np.count_nonzero( (P[:,0,0]>=sigma).astype(int))
# plt.plot(lon,P[:,0,20],'r')
plt.plot(P[:,0,0:500])
plt.plot(sigma)

